In [60]:
# import pandas as pd
# import re
# from nltk.corpus import stopwords
# from gensim.models import Word2Vec
# import multiprocessing

# df = pd.read_csv("text-generator.csv",  encoding='cp1252')

# def clean_data(text):
#     text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
#     text = re.sub(r'[\\/×\^\]\[÷]', '', text)
#     return text
# def change_lower(text):
#     text = text.lower()
#     return text

# stopwords_list = stopwords.words("english")
# def remover(text):
#     text_tokens = text.split(" ")
#     final_list = [word for word in text_tokens if not word in stopwords_list]
#     text = ' '.join(final_list)
# #     print(text)
#     return text

# def get_w2vdf(df):
#     w2v_df = pd.DataFrame(df["Text"]).values.tolist()
#     for i in range(len(w2v_df)):
#         w2v_df[i] = w2v_df[i][0].split(" ")
#     return w2v_df

# def train_w2v(w2v_df):
#     cores = multiprocessing.cpu_count()
#     w2v_model = Word2Vec(min_count=4,
#                          window=4,
# #                          size=300, 
#                          alpha=0.03, 
#                          min_alpha=0.0007, 
#                          sg = 1,
#                          workers=cores-1)
    
#     w2v_model.build_vocab(w2v_df, progress_per=10000)
#     w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
#     return w2v_model

# df[["Text"]] = df[["Text"]].astype(str)
# df["Text"] = df["Text"].apply(change_lower)
# df["Text"] = df["Text"].apply(clean_data)
# df["Text"] = df["Text"].apply(remover)

# w2v_df = get_w2vdf(df)
# w2v_model = train_w2v(w2v_df)
# # print(w2v_df)

In [61]:
from gensim.models import Word2Vec
import nltk
from nltk import word_tokenize

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0
for review in df['Text']:
  sents=tokenizer.tokenize(review.strip())
  sum+=len(sents)
  for sent in sents:
#     cleaned_sent=clean_reviews(sent)
    sentences.append(sent.split()) # can use word_tokenize also.
print(sum)
print(len(sentences))  # total no of sentences

365
365


In [58]:
w2v_model.wv.get_vector('information')

array([ 0.12119494,  0.34125713,  0.20713627,  0.1709707 , -0.18127331,
        0.03996315,  0.29467914,  0.28692764, -0.36901525, -0.00207038,
        0.358314  ,  0.16404672, -0.17583953,  0.06934   ,  0.05710686,
       -0.02452595, -0.11645096, -0.01343467, -0.12369075, -0.37669498,
        0.20750453, -0.09735931,  0.02207841,  0.01602149, -0.01123085,
        0.01394659, -0.07664207, -0.06457649,  0.02797045,  0.30245718,
       -0.17128928,  0.03568438, -0.01372833,  0.3231812 , -0.33298925,
       -0.15583357, -0.01084761,  0.05087924,  0.04332731, -0.14096728,
       -0.00066474, -0.09403183, -0.03481948,  0.08074769,  0.39247745,
       -0.0908554 , -0.27030477,  0.3244035 ,  0.34192628,  0.01945674,
        0.08710952, -0.21660838, -0.05753756, -0.2152091 , -0.25447187,
       -0.11703781,  0.17809491,  0.0218898 ,  0.22927998, -0.15148245,
       -0.03188246, -0.06462417, -0.05065012,  0.10870004, -0.39706904,
        0.13400418,  0.03219141,  0.20778579, -0.21537575,  0.13

In [26]:
w2v_model.wv.most_similar('process')

[('government', 0.4234906733036041),
 ('transactions', 0.4179198741912842),
 ('family', 0.3950233459472656),
 ('needs', 0.38710522651672363),
 ('numbers', 0.3843950927257538),
 ('support', 0.37131965160369873),
 ('disclose', 0.3637078106403351),
 ('authorities', 0.359550803899765),
 ('behalf', 0.35748448967933655),
 ('merchants', 0.35155895352363586)]

In [27]:
w2v_model.wv.similarity('information','data')

0.3079453

In [30]:
print("The no of words :",len(w2v_model.wv.index_to_key))

The no of words : 657


In [32]:
vocab=list(w2v_model.wv.index_to_key)

In [33]:
word_vec_dict={}
for word in w2v_model.wv.index_to_key:
    word_vec_dict[word]=w2v_model.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  657


In [36]:
maxi=-1
for i,rev in enumerate(df['Text']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

147


In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer 

tok = Tokenizer()
tok.fit_on_texts(df['Text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['Text'])

In [43]:
max_rev_len=147  # max lenght of a texte
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100 # embedding dimension as choosen in word2vec constructor

In [44]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape

(365, 147)

In [62]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
    embed_vector=word_vec_dict.get(word)
    if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

In [46]:
print(embed_matrix[14])

[ 0.03509673  0.65169078 -0.48518682  0.43348512  0.47571629 -0.14120139
 -0.5007906  -0.08653297 -0.48139405 -0.34685338 -0.1391917   0.2740941
 -0.84675139 -0.32729667  0.57380289  0.5494774   0.13669755 -0.20296624
 -0.67501092 -0.67814964  0.37982097 -0.27690583  0.36609346 -0.31082335
  0.26088044  0.07762034 -0.13885926 -0.1397426  -0.48620063  0.73688012
  0.25937539  0.55652648  0.01769175  0.13391206 -0.01005688  0.05293911
 -0.24077915  0.8105945   0.19337034 -0.12585549 -0.33557549  0.2433756
  0.03117934 -0.11962032  0.51375318 -0.03263777  0.23306128 -0.08587485
  0.87020373 -0.21787582 -0.00667506 -0.3920128  -0.28369379 -0.13557704
  0.1839419   0.47875446  0.62673849  0.31227276 -0.39770517  0.20264953
  0.03453618  0.50732619 -0.10847239  0.08083101 -0.14297186 -0.3182106
  0.09078025 -0.05982817 -0.23254313  0.29192856  0.0241233  -0.04972333
 -0.29940939 -0.12638225  0.3599107  -0.21960202 -0.00275448  0.3285495
  0.37006816  0.0878973   0.00571305  0.24005665 -0.353

In [57]:

input_texts=[]
target_texts=[]

# Pass the input records and taret records
for in_txt,tr_txt in zip(input_data,target_data):
    in_words= in_txt
    input_texts+= [' '.join(in_words)]
    input_words+= in_words
    # Add 'start' at start and 'end' at end of text
    tr_words= clean("start "+tr_txt+" end")
    target_texts+= [' '.join(tr_words)]
    target_words+= tr_words

NameError: name 'input_words' is not defined

In [63]:
from sklearn.model_selection import train_test_split
# import keras
from tensorflow.keras.utils import to_categorical


# input_data = df.loc[:,'Text']
target_data = df.loc[:,'Summary']
# target_data=keras.utils.to_categorical(df['Summary'])  # one hot target as required by NN.

x_train,x_test,y_train,y_test=train_test_split(pad_rev,target_texts,test_size=0.1,random_state=40)
x_train

ValueError: Found input variables with inconsistent numbers of samples: [365, 0]